In [1]:
import time
start_time = time.time()
from subprocess import check_output
print(check_output(["ls", "./"]).decode('utf8'))

best_model.hdf5
crawl-300d-2M.vec
fasttext-crawl-300d-2m.zip
glove.6B.50d.txt
glove.840B.300d.txt
glove.840B.300d.txt.zip
Guru.ipynb
New Guru- Bi-GRU-CNN-Pooling.ipynb
notebook.ipynb
Replacers.py
sample_submission.csv
submission.csv
submission-Guru-10.csv
submission-Guru-11.csv
submission-Guru-12.csv
submission-Guru-13.csv
submission_version-1(LSTM).csv
submission_version-2(LSTM+tanh+softmax).csv
submission_version-3_GRU.csv
submission_version-4_CNN.csv
submission_version-5_GRU.csv
submission_version-6_LSTM+ConvNet.csv
submission_version-7+GRU_Pre.csv
submission_version-8+GRU_withoutPre.csv
test.csv
Test_witout_stop.csv
Toxic_Prediction_with_keras.ipynb
Toxic_Prediction_with_keras_VERSION-01.ipynb
Toxic_Solution.ipynb
train.csv
Train_without_stop.csv



In [2]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load in 

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)
from sklearn.model_selection import train_test_split
from sklearn.metrics import roc_auc_score
from keras.models import Model
from keras.layers import Input, Dense, Embedding, SpatialDropout1D, concatenate
from keras.layers import GRU, Bidirectional, GlobalAveragePooling1D, GlobalMaxPooling1D
from keras.preprocessing import text, sequence
from keras.callbacks import Callback

from sklearn.model_selection import train_test_split
import sys, os, re, csv, codecs, numpy as np, pandas as pd
np.random.seed(32)
os.environ["OMP_NUM_THREADS"] = "4"
from keras.preprocessing.text import Tokenizer
from keras.preprocessing.sequence import pad_sequences
from keras.layers import Dense, Input, LSTM, Embedding, Dropout, Activation, Conv1D
from keras.layers import Bidirectional, GlobalMaxPool1D, MaxPooling1D, Add, Flatten
from keras.layers import GlobalAveragePooling1D, GlobalMaxPooling1D, concatenate, SpatialDropout1D
from keras.models import Model, load_model
from keras import initializers, regularizers, constraints, optimizers, layers, callbacks
from keras import backend as K
from keras.engine import InputSpec, Layer

# Input data files are available in the "../input/" directory.
# For example, running this (by clicking run or pressing Shift+Enter) will list the files in the input directory

import os
os.environ['OMP_NUM_THREADS'] = '4'
print(os.listdir("../"))

# Any results you write to the current directory are saved as output.

Using TensorFlow backend.


['Toxic', 'kaggle-competitions']


In [3]:
def standardize_text(df, text_field):
    df[text_field] = df[text_field].str.replace(r"http\S+", "LINK")
    df[text_field] = df[text_field].str.replace(r"http", "LINK")
    df[text_field] = df[text_field].str.replace(r"xml\\S+", "LINK")
    df[text_field] = df[text_field].str.replace(r"@\S+", "")
    df[text_field] = df[text_field].str.replace(r"[^A-Za-z0-9(),!?@\'\`\"\_\n]", " ")
    
    df[text_field] = df[text_field].str.replace(r"\\b(a|e)w+\\b", "AWWWW")
    df[text_field] = df[text_field].str.replace(r"\\b(y)a+\\b", "YAAAA")
    df[text_field] = df[text_field].str.replace(r"\\b(w)w+\\b", "WWWWW")
    df[text_field] = df[text_field].str.replace(r"\\b(b+)?((h+)((a|e|i|o|u)+)(h+)?){2,}\\b", "HAHEHI")
    df[text_field] = df[text_field].str.replace(r"\\b(b+)?(((a|e|i|o|u)+)(h+)((a|e|i|o|u)+)?){2,}\\b", "HAHEHI")
    df[text_field] = df[text_field].str.replace(r"\\b(m+)?(u+)?(b+)?(w+)?((a+)|(h+))+\\b", "HAHEHI")
    df[text_field] = df[text_field].str.replace(r"\\b((e+)(h+))+\\b", "HAHEHI",)
    df[text_field] = df[text_field].str.replace(r"\\b((e+)(h+))+\\b", "HAHEHI",)
    df[text_field] = df[text_field].str.replace(r"\\b((o+)(h+))+\\b", "HAHEHI")
    df[text_field] = df[text_field].str.replace(r"\\b((h+)(o+))+\\b", "HAHEHI")
    df[text_field] = df[text_field].str.replace(r"\\b((l+)(a+))+\\b", "LALALA")
    df[text_field] = df[text_field].str.replace(r"(w+)(o+)(h+)(o+)", "WOHOO")
    df[text_field] = df[text_field].str.replace(r"\\b(d?(u+)(n+)?(h+))\\b", "UUUHHH")
    df[text_field] = df[text_field].str.replace(r"\\b(a+)(r+)(g+)(h+)\\b", "ARGH")
    df[text_field] = df[text_field].str.replace(r"\\b(a+)(w+)(h+)\\b", "AAAWWHH")
    df[text_field] = df[text_field].str.replace(r"\\b(p+)(s+)(h+)\\b", "SHHHHH")
    df[text_field] = df[text_field].str.replace(r"\\b((s+)(e+)?(h+))+\\b", "SHHHHH" )                                               
    df[text_field] = df[text_field].str.replace(r"\\b(s+)(o+)\\b", "")
    df[text_field] = df[text_field].str.replace(r"\\b(h+)(m+)\\b", "HHMM")
    df[text_field] = df[text_field].str.replace(r"\\b((b+)(l+)(a+)(h+)?)+\\b", "BLABLA")
    df[text_field] = df[text_field].str.replace(r"\\b((y+)(e+)(a+)(h+)?)+\\b", "YEAH")                                               
    df[text_field] = df[text_field].str.replace(r"\\b((z+)?(o+)(m+)(f+)?(g+))+\\b", "OMG"  )
    df[text_field] = df[text_field].str.replace(r"\\b((y+)(e+)(a+)(h+)?)+\\b", "YEAH"   )
    df[text_field] = df[text_field].str.replace(r"aa(a+)", "a")
    df[text_field] = df[text_field].str.replace(r"ee(e+)", "e")
    df[text_field] = df[text_field].str.replace(r"i(i+)", "i")
    df[text_field] = df[text_field].str.replace(r"oo(o+)", "o")
    df[text_field] = df[text_field].str.replace(r"uu(u+)", "u")
    df[text_field] = df[text_field].str.replace(r"\\b(u(u+))\\b", "u")
    df[text_field] = df[text_field].str.replace(r"y(y+)", "y")
    df[text_field] = df[text_field].str.replace(r"hh(h+)", "h")
    df[text_field] = df[text_field].str.replace(r"gg(g+)", "g")
    df[text_field] = df[text_field].str.replace(r"tt(t+)\\b", "t")
    df[text_field] = df[text_field].str.replace(r"(tt(t+))", "tt")
    df[text_field] = df[text_field].str.replace(r"mm(m+)", "m")
    df[text_field] = df[text_field].str.replace(r"ff(f+)", "f")
    df[text_field] = df[text_field].str.replace(r"cc(c+)", "c")
    df[text_field] = df[text_field].str.replace(r"\\b(kkk)\\b", "KKK")
    df[text_field] = df[text_field].str.replace(r"\\b(pkk)\\b", "PKK")
    df[text_field] = df[text_field].str.replace(r"kk(k+)", "kk")
    df[text_field] = df[text_field].str.replace(r"fukk", "fuck")
    df[text_field] = df[text_field].str.replace(r"k(k+)\\b", "k")
    df[text_field] = df[text_field].str.replace(r"f+u+c+k+\\b", "fuck")
    df[text_field] = df[text_field].str.replace(r"((a+)|(h+)){3,}", "HAHEHI")
    df[text_field] = df[text_field].str.replace(r"yeah", "YEAH")
    df[text_field] = df[text_field].str.replace(r"(?<=\\b\\w)\\s(?=\\w\\b)", "")
    df[text_field] = df[text_field].str.replace(r"@\\w+", " ")
    df[text_field] = df[text_field].str.replace(r"[^[:graph:]]", " ")
    df[text_field] = df[text_field].str.replace(r"[[:punct:]]", " ")
    df[text_field] = df[text_field].str.replace(r"[[:digit:]]", " ")
    df[text_field] = df[text_field].str.replace(r"\\s+", " ")
    df[text_field] = df[text_field].str.replace(r"((lol)(o?))+\\b", "LOL")
    df[text_field] = df[text_field].str.replace(r"n ig ger", "nigger")
    df[text_field] = df[text_field].str.replace(r"nig ger", "nigger")
    df[text_field] = df[text_field].str.replace(r"s hit", "shit")
    df[text_field] = df[text_field].str.replace(r"g ay", "gay")
    df[text_field] = df[text_field].str.replace(r"f ag got", "faggot")
    df[text_field] = df[text_field].str.replace(r"c ock", "cock")
    df[text_field] = df[text_field].str.replace(r"cu nt", "cunt")
    df[text_field] = df[text_field].str.replace(r"idi ot", "idiot")
    df[text_field] = df[text_field].str.replace(r"(?<=\\b(fu|su|di|co|li))\\s(?=(ck)\\b)", "")
    df[text_field] = df[text_field].str.replace(r"(?<=\\w(ck))\\s(?=(ing)\\b)", "")
    df[text_field] = df[text_field].str.replace(r"\\b(.)\\1+\\b", " ")
    df[text_field] = df[text_field].str.replace(r"((lol)(o?))+", "LOL")
    df[text_field] = df[text_field].str.replace(r"(?<=\\b(fu|su|di|co|li))\\s(?=(ck)\\b)", "")
    df[text_field] = df[text_field].str.replace(r"(?<=\\w(ck))\\s(?=(ing)\\b)", "")
    df[text_field] = df[text_field].str.replace(r"(?<=\\w(uc))\\s(?=(ing)\\b)", "")
    df[text_field] = df[text_field].str.replace(r"(?<=\\b(fu|su|di|co|li))\\s(?=(ck)\\w)", "")
    df[text_field] = df[text_field].str.replace(r"cocksu cking", "cock sucking")
    df[text_field] = df[text_field].str.replace(r"du mbfu ck", "dumbfuck")
    df[text_field] = df[text_field].str.replace(r"cu nt", "cunt")
    df[text_field] = df[text_field].str.replace(r"(?<=\\b(fu|su|di|co|li))\\s(?=(k)\\w)", "c")
    df[text_field] = df[text_field].str.replace(r"\\s+", " ")
    df[text_field] = df[text_field].str.replace(r"\\s*$", "")
    df[text_field] = df[text_field].str.replace(r"^\\s+", "")
    df[text_field] = df[text_field].str.replace(r"\\b(.)\\1+\\b", " ")
    df[text_field] = df[text_field].str.replace(r"\\W*\\b([a-h|j-z])\\b\\W*", " ")
    df[text_field] = df[text_field].str.replace(r"\\s+", " ")
    df[text_field] = df[text_field].str.replace(r"\\s*$", "")
    df[text_field] = df[text_field].str.replace(r"^\\s+", "")
        
        
    df[text_field] = df[text_field].str.replace(r"@", "at")
    df[text_field] = df[text_field].str.replace(r"&", "and")
    df[text_field] = df[text_field].str.replace(r"$", "dollar")
    
    df[text_field] = df[text_field].str.replace(r"what's", "what is")
    df[text_field] = df[text_field].str.replace(r"\'ve", " have")
    df[text_field] = df[text_field].str.replace(r"can't", " can not")
    df[text_field] = df[text_field].str.replace(r"cannot", " can not")
    df[text_field] = df[text_field].str.replace(r"n't", " not")
    df[text_field] = df[text_field].str.replace(r"i'm", "i am")
    df[text_field] = df[text_field].str.replace(r"\'re"," are")
    df[text_field] = df[text_field].str.replace(r"\'d", " would")
    df[text_field] = df[text_field].str.replace(r"\'ll", " will")
    
    df[text_field] = df[text_field].str.lower()
    return df

In [4]:
#EMBEDDING_FILE = f'glove.840B.300d.txt'

EMBEDDING_FILE = f'crawl-300d-2M.vec'

In [5]:
train = pd.read_csv("train.csv")
test = pd.read_csv("test.csv")

In [6]:
train["comment_text"].fillna(" ")
test["comment_text"].fillna(" ")
train = standardize_text(train,"comment_text")
test = standardize_text(test,"comment_text")

In [7]:
train_x = train["comment_text"].values
train_y = train[["toxic", "severe_toxic", "obscene", "threat", "insult", "identity_hate"]].values
test_x = test["comment_text"].values

In [8]:
embed_size = 300
#max_features = 100000
max_features = 100500
#max_len = 150
max_len = 175

In [9]:
tokenizer = text.Tokenizer(num_words=max_features)
tokenizer.fit_on_texts(list(train_x) + list(test_x))

In [10]:
train_x = tokenizer.texts_to_sequences(train_x)
test_x = tokenizer.texts_to_sequences(test_x)

In [11]:
train_x = sequence.pad_sequences(train_x, maxlen=max_len)
test_x = sequence.pad_sequences(test_x, maxlen=max_len)

In [12]:
def get_coefs(word, *arr): 
    return word, np.asarray(arr, dtype='float32')

In [13]:
embeddings_index = dict(get_coefs(*o.rstrip().rsplit(' ')) for o in open(EMBEDDING_FILE))

In [14]:
word_index = tokenizer.word_index

In [15]:
nb_words = min(max_features, len(word_index))

In [16]:
embedding_matrix = np.zeros((nb_words, embed_size))

In [17]:
for word, i in word_index.items():
    if i >= max_features: continue
    embedding_vector = embeddings_index.get(word)
    if embedding_vector is not None: embedding_matrix[i] = embedding_vector

In [18]:
import logging
from sklearn.metrics import roc_auc_score
from keras.callbacks import Callback

class RocAucEvaluation(Callback):
    def __init__(self, validation_data=(), interval=1):
        super(Callback, self).__init__()

        self.interval = interval
        self.X_val, self.y_val = validation_data

    def on_epoch_end(self, epoch, logs={}):
        if epoch % self.interval == 0:
            y_pred = self.model.predict(self.X_val, verbose=0)
            score = roc_auc_score(self.y_val, y_pred)
            print("\n ROC-AUC - epoch: {:d} - score: {:.6f}".format(epoch+1, score))

In [19]:
#X_train, X_valid, Y_train, Y_valid = train_test_split(train, y, test_size = 0.1)
#X_train, X_valid, Y_train, Y_valid = train_test_split(train_x, train_y, test_size = 0.1, random_state=233)
X_train, X_valid, Y_train, Y_valid = train_test_split(train_x, train_y, test_size = 0.05, random_state=233)

In [20]:
from keras.optimizers import Adam, RMSprop
from keras.callbacks import EarlyStopping, ModelCheckpoint, LearningRateScheduler
from keras.layers import GRU, BatchNormalization, Conv1D, MaxPooling1D

file_path = "best_model.hdf5"
check_point = ModelCheckpoint(file_path, monitor = "val_loss", verbose = 1,
                              save_best_only = True, mode = "min")
ra_val = RocAucEvaluation(validation_data=(X_valid, Y_valid), interval = 1)
early_stop = EarlyStopping(monitor = "val_loss", mode = "min", patience = 5)



In [21]:
def build_model(lr = 0.0, lr_d = 0.0, units = 0, dr = 0.0):
    inp = Input(shape = (max_len,))
    x = Embedding(max_features, embed_size, weights = [embedding_matrix], trainable = False)(inp)
    x = SpatialDropout1D(dr)(x)

    #x = Bidirectional(GRU(units, return_sequences = True))(x)
    x = Bidirectional(LSTM(units, return_sequences = True))(x)
    #x = Conv1D(64, kernel_size = 2, padding = "valid", kernel_initializer = "he_uniform")(x)
    x = Conv1D(64, kernel_size = 3, padding = "valid", kernel_initializer = "glorot_uniform")(x)
    avg_pool = GlobalAveragePooling1D()(x)
    max_pool = GlobalMaxPooling1D()(x)
    x = concatenate([avg_pool, max_pool])

    x = Dense(6, activation = "sigmoid")(x)
    model = Model(inputs = inp, outputs = x)
    model.compile(loss = "binary_crossentropy", optimizer = Adam(lr = lr, decay = lr_d), metrics = ["accuracy"])
    history = model.fit(X_train, Y_train, batch_size = 128, epochs = 4, validation_data = (X_valid, Y_valid), 
                        verbose = 1, callbacks = [ra_val, check_point, early_stop])
    model = load_model(file_path)
    return model

In [22]:
model = build_model(lr = 1e-3, lr_d = 0, units = 128, dr = 0.2)
model.summary()

Train on 151592 samples, validate on 7979 samples
Epoch 1/4
151552/151592 [============================>.] - ETA: 0s - loss: 0.0556 - acc: 0.9802
 ROC-AUC - epoch: 1 - score: 0.986229
Epoch 00001: val_loss improved from inf to 0.04945, saving model to best_model.hdf5
151592/151592 [==============================] - 3186s 21ms/step - loss: 0.0556 - acc: 0.9802 - val_loss: 0.0494 - val_acc: 0.9817
Epoch 2/4
151552/151592 [============================>.] - ETA: 0s - loss: 0.0427 - acc: 0.9836
 ROC-AUC - epoch: 2 - score: 0.987662
Epoch 00002: val_loss improved from 0.04945 to 0.04329, saving model to best_model.hdf5
151592/151592 [==============================] - 3674s 24ms/step - loss: 0.0427 - acc: 0.9836 - val_loss: 0.0433 - val_acc: 0.9835
Epoch 3/4
151552/151592 [============================>.] - ETA: 0s - loss: 0.0401 - acc: 0.9844
 ROC-AUC - epoch: 3 - score: 0.988943
Epoch 00003: val_loss did not improve
151592/151592 [==============================] - 3878s 26ms/step - loss: 0.0

In [23]:
y_pred = model.predict(test_x, batch_size=1024, verbose=1)

153164/153164 [==============================] - 1325s 9ms/step


In [24]:
submission = pd.read_csv('sample_submission.csv')

In [25]:
submission[["toxic", "severe_toxic", "obscene", "threat", "insult", "identity_hate"]] = y_pred

In [26]:
submission.to_csv('submission-Guru_LSTM-14.csv', index=False)

In [27]:
model.summary()

__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_1 (InputLayer)            (None, 175)          0                                            
__________________________________________________________________________________________________
embedding_1 (Embedding)         (None, 175, 300)     30150000    input_1[0][0]                    
__________________________________________________________________________________________________
spatial_dropout1d_1 (SpatialDro (None, 175, 300)     0           embedding_1[0][0]                
__________________________________________________________________________________________________
bidirectional_1 (Bidirectional) (None, 175, 256)     439296      spatial_dropout1d_1[0][0]        
__________________________________________________________________________________________________
conv1d_1 (

In [28]:
print("[{}] Completed!".format(time.time() - start_time))

[16153.535946130753] Completed!
